# Homework 2: Naive Bayes Email Spam Filter

In this homework, we will learn how to implement the Naive Bayes classifier in order to create a simple email spam filter. This spam filter will be trained by test_emails, which will be given by a vector of tuples (emails, spam/nospam). For each tuple the first entry is a string ("email"), and the second entry is 0 or 1, depending whether the email contains spam words.

In [1]:
dictionary = ['hello', 'students', 'please', 'learn', 'for', 'the', 'exam', 'buy', 'drugs', 'today', 'sun', 'is', 'shining', 'in', 'nagoya', 'lets', 'sell', 'how', 'are', 'you', 'today?', 'do', 'your', 'homework', 'want', 'free', 'solutions?', 'hey', 'always', 'ask', 'questions', 'if','have', 'any', 'math', 'not', 'good', 'submit', 'pay'] 

test_emails=[
             ["hello students, please learn for the exam", 0],
             ["hello students, please buy drugs", 1],    
             ["hello, today the sun is shining in nagoya", 0],
             ["lets sell drugs in nagoya", 1],
             ["today learn drugs", 1],
             ["how are you today?", 0],
             ["hello students, please do your homework", 0],
             ["hello, do you want free homework solutions?", 1],
             ["hey students, please always ask questions if you have any", 0],
             ["math is not good", 1],
             ["math is good", 0],
             ["please submit your homework", 0],
             ["please buy questions", 1],
             ["please pay for the exam", 1]          
             ]

The feature space for our spam filter will be $\mathcal{X}=\{0,1\}^d$, where $d$ denotes the number of words in the dictionary. For a feature (email) $x \in \mathcal{X}$ the entry $x_i$ for $i=1,\dots,d$ is $1$ if the $i$-th word of the dictionary is contained in the email and $0$ otherwise.  

# **Exercise 1**
Implement a function which creates a feature vector out of an email and a function which creates a training set out of test emails. 

You would need to figure out whether a sentence contains a word, and there are functions in Python that could determine whether a string contains another string. You can consult documentation (and Google) to find out.

In [2]:
import numpy as np
def email_to_feature(dict, email):
  d = len(dict)
  x = np.zeros(d)
  for j in range(d):
    if dict[j] in email:
      x[j] = 1
  return x

def emails_to_test_samples(dict, test_emails):
  d = len(dict)
  l = len(test_emails)
  x = np.zeros((l,d))
  y = np.zeros(l)
  for j in range(l):
    x[j] = email_to_feature(dict, test_emails[j][0])
    y[j] = test_emails[j][1] #y[j] is 0(not spam) or 1(spam). 
  return x, y

In [3]:
print(emails_to_test_samples(dictionary, test_emails)[0])

[[1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.
  1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0

 **Recall from Lecture 6:**

Given a training set  $\mathcal{T} = \left( (x^{(1)}, y^{(1)}) , \dots, (x^{(n)}, y^{(n)})   \right)$ we calculate for $i=1,\dots,d$ the following numbers
\begin{align*}
\phi_{i\mid y=1} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=1 ) }{2+\sum_{j=1}^n I(y^{(j)}=1)}\,,\\
	\phi_{i\mid y=0} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=0 )}{2+\sum_{j=1}^n I(y^{(j)}=0)}\,,\\
		\phi_{y=1} &= \frac{1+\sum_{j=1}^n I(y^{(j)} = 1)}{2+n} \,.
\end{align*}
Here $I$ denoted the indicator function. We used the laplace smoothing (thats why we have the $1+$ and $2+$) in order to make sure that we will not assume probability $0$ for unknown words.

Now assume we get a new feature (i.e. someone sends us an email) $x \in \{0,1\}^d$. Then we can calculate for each word $i=1,\dots,d$ the probabilities
\begin{align*}
P(x_i = 1 \mid y=1) &= \phi_{i\mid y=1}\,,\qquad &&P(x_i = 1 \mid y=0) = \phi_{i\mid y=0} \,,\\
P(x_i = 0 \mid y=1) &= 1- \phi_{i\mid y=1}\,,\qquad &&P(x_i = 0 \mid y=0) = 1-\phi_{i\mid y=0} \,. \\
\end{align*}

By the Naive Bayes assumption we have for $x \in \{0,1\}^d$
\begin{align*}
P(x \mid y)  = \prod_{i=1}^d P(x_j \mid y)\,.
\end{align*}
The probability of the new email being spam is then
\begin{align*}
P(y=1 \mid x) &= \frac{ P(x \mid y=1) P(y=1)}{P(x)}\\
&= \frac{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1}}{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1} + \prod_{i=1}^d P(x_i \mid y = 0) (1-\phi_{y=1})}\,.
\end{align*}


# **Exercise 2:** 
Use the above explanation of the Naive Bayes Spam filter and implement a function which gives the probability of an email being spam given the trainings email above. 


In [56]:
# You can implement the above algorithm in any way you want.
# One possible way:
# - Calculate the phis here
# - Write functions for the probability P(x|y) depending on these phis
# - Use this function in the function spam_percentage 

def phi(x, y, dict, test_emails):
  d = len(dict)
  n = len(test_emails)
  phi_y1 = np.zeros(d)
  phi_y0 = np.zeros(d)
  #The following value "denom" is independent of i, so we calculate it first.
  denom1 = 2
  denom2 = 2
  for j in range(n):
    if y[j] == 1:
      denom1 += 1
    if y[j] == 0:
      denom2 += 1
  #Then we calculate the numerators.
  
  for i in range(d): 
    num1 = 1
    num2 = 1
    for j in range(n):
      if x[j][i] == 1 and y[j] == 1:
        num1 += 1
      if x[j][i] == 1 and y[j] == 0:
        num2 += 1
    phi_y1[i] = num1/denom1
    phi_y0[i] = num2/denom2
  #We finished calculating phi_y0 and phi_y1.

  num3 = 1
  for j in range(n):
    if y[j] == 1:
      num3 += 1
  phi = num3/(2+n)
  
  return phi_y1, phi_y0, phi



  

def spam_percentage(dict, test_emails, email):
  d = len(dict)
  n = len(test_emails)
  x = emails_to_test_samples(dict, test_emails)[0]
  y = emails_to_test_samples(dict, test_emails)[1]
  #Let us calculate P(y=1|email). Let us call the feature vector of email data.
  data = email_to_feature(dict, email)

  num = phi(x, y, dict, test_emails)[2]
  denom1 = 1
  denom2 = 1 - phi(x, y, dict, test_emails)[2]
  for i in range(d):
    if data[i] == 1:
      num *= phi(x, y, dict, test_emails)[0][i]
    if data[i] == 0:
      num *= 1 - phi(x, y, dict, test_emails)[0][i]
  denom1 = num
  for i in range(d):
    if data[i] == 1:
      denom2 *= phi(x, y, dict, test_emails)[1][i]
    if data[i] == 0:
      denom2 *= 1 - phi(x, y, dict, test_emails)[1][i]
  return num/(denom1 + denom2)


Test your spam filter with the following email.


In [74]:
email="the sun is shining. buy drgus now"
email2 = "hello students, please buy drugs"
print(spam_percentage(dictionary, test_emails, email2))
print(spam_percentage(new_dictionary, test_emails, email2))

0.9680926494918458
0.963699919333154



# **Exercise 3**
Extend your spamfilter by creating a dynamical dictionary. Instead of starting with a fixed dictionary, you should now create a dictionary out of a list of emails. 

Write a function `create_dictionary(emails)` which returns a dictionary created from a list of emails (Give as an array of arrays `[text, spam\nospam]`). Make sure that you do not include words more than once into the dictionary.
To implement this function you should look up the function `split()` for a string in Python. To take care of the symbols "." and "," you can use the `replace()` function of a string.

In [72]:
def create_dictionary(emails):
  new_dict = []
  n = len(emails)
  for i in range(n):
    sentence = emails[i][0].split()
    for j in range(len(sentence)):
      sentence[j] = sentence[j].replace("?", "")
      sentence[j] = sentence[j].replace("!", "")
      sentence[j] = sentence[j].replace("'", "")
      sentence[j] = sentence[j].replace(".", "")
      sentence[j] = sentence[j].replace(",", "")
      if sentence[j] not in new_dict:
        new_dict.append(sentence[j])
  return new_dict

In [71]:
new_dictionary = create_dictionary(test_emails)
new_dictionary

['hello',
 'students',
 'please',
 'learn',
 'for',
 'the',
 'exam',
 'buy',
 'drugs',
 'today',
 'sun',
 'is',
 'shining',
 'in',
 'nagoya',
 'lets',
 'sell',
 'how',
 'are',
 'you',
 'do',
 'your',
 'homework',
 'want',
 'free',
 'solutions',
 'hey',
 'always',
 'ask',
 'questions',
 'if',
 'have',
 'any',
 'math',
 'not',
 'good',
 'submit',
 'pay']